## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-14-22-51-31 +0000,nypost,White House rips NYC Councilman Chi Ossé for c...,https://nypost.com/2026/02/14/us-news/white-ho...
1,2026-02-14-22-50-41 +0000,nypost,Puppy love: NYC canine cafe offers Valentine’s...,https://nypost.com/2026/02/14/us-news/puppy-lo...
2,2026-02-14-22-50-33 +0000,nyt,"No miracle, but feisty Denmark gives U.S. a test.",https://www.nytimes.com/2026/02/14/world/no-mi...
3,2026-02-14-22-47-21 +0000,nypost,Over two-dozen cars involved in hectic pileup ...,https://nypost.com/2026/02/14/us-news/over-two...
4,2026-02-14-22-44-16 +0000,nypost,"Zelensky begs Trump ‘give us a ceasefire,’ vow...",https://nypost.com/2026/02/14/world-news/zelen...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2462/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
8,trump,20
125,ice,13
61,new,12
90,rubio,10
79,europe,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
93,2026-02-14-15-01-39 +0000,nypost,Marco Rubio calls on Europe to join new world ...,https://nypost.com/2026/02/14/world-news/marco...,67
154,2026-02-14-06-47-49 +0000,nypost,White House unveils cheeky Valentine’s Day car...,https://nypost.com/2026/02/14/us-news/white-ho...,50
59,2026-02-14-17-47-57 +0000,wapo,"Rubio says U.S., Europe ‘belong together,’ des...",https://www.washingtonpost.com/politics/2026/0...,46
77,2026-02-14-16-29-44 +0000,nypost,Pam Bondi was mocked for calling out crime in...,https://nypost.com/2026/02/14/us-news/man-dead...,46
35,2026-02-14-19-46-33 +0000,nyt,"In Munich, Europe Says It Is ‘De-risking’ From...",https://www.nytimes.com/2026/02/14/world/europ...,44


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
93,67,2026-02-14-15-01-39 +0000,nypost,Marco Rubio calls on Europe to join new world ...,https://nypost.com/2026/02/14/world-news/marco...
77,46,2026-02-14-16-29-44 +0000,nypost,Pam Bondi was mocked for calling out crime in...,https://nypost.com/2026/02/14/us-news/man-dead...
71,40,2026-02-14-17-00-14 +0000,bbc,Russia killed opposition leader Alexei Navalny...,https://www.bbc.com/news/articles/cwyk4lz4e3eo...
154,40,2026-02-14-06-47-49 +0000,nypost,White House unveils cheeky Valentine’s Day car...,https://nypost.com/2026/02/14/us-news/white-ho...
115,38,2026-02-14-12-00-00 +0000,startribune,Minneapolis has 950 surveillance cameras. Some...,https://www.startribune.com/minneapolis-survei...
92,36,2026-02-14-15-08-20 +0000,nypost,Savannah Guthrie’s husband Michael Feldman see...,https://nypost.com/2026/02/14/us-news/savannah...
52,33,2026-02-14-18-05-00 +0000,nypost,"Maniac pushes woman onto NYC subway tracks, pu...",https://nypost.com/2026/02/14/us-news/women-pu...
70,33,2026-02-14-17-02-08 +0000,nypost,200K protesters fight Iran’s Islamic Republic ...,https://nypost.com/2026/02/14/world-news/200k-...
158,30,2026-02-14-05-25-21 +0000,nyt,Casey Wasserman Will Sell Entertainment Agency...,https://www.nytimes.com/2026/02/13/us/casey-wa...
173,29,2026-02-14-03-07-45 +0000,nypost,Ex-ESPN star Sarah Spain felt ‘ill’ sitting ne...,https://nypost.com/2026/02/13/us-news/ex-espn-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
